In [2]:
!nvidia-smi

Sat Feb 22 02:00:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.16              Driver Version: 572.16         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               On  |   00000000:15:00.0  On |                  Off |
| 30%   45C    P2             66W /  230W |    5598MiB /  24564MiB |     35%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install flash-attn --no-build-isolation

In [1]:
from PIL import Image 
from transformers import AutoProcessor, AutoModelForImageTextToText 
import torch 

model_path = "HuggingFaceTB/SmolVLM2-2.2B-Instruct" 
processor = AutoProcessor.from_pretrained(model_path) 
model = AutoModelForImageTextToText.from_pretrained(model_path, torch_dtype=torch.bfloat16, _attn_implementation="flash_attention_2").to("cuda") 

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.06it/s]


Sat Feb 22 01:45:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.16              Driver Version: 572.16         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               On  |   00000000:15:00.0  On |                  Off |
| 30%   43C    P8             16W /  230W |    5556MiB /  24564MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
messages = [{ "role": "user", "content": [ {"type": "video", "path": "Music.mp4"}, {"type": "text", "text": "Describe this video in detail"} ] },]

inputs = processor.apply_chat_template( messages, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt", ).to(model.device, torch.bfloat16) 



In [4]:
generated_ids = model.generate(**inputs, do_sample=False, max_new_tokens=564) 

In [5]:
generated_texts = processor.batch_decode( generated_ids, skip_special_tokens=True, )

In [6]:
generated_texts

['User: You are provided the following series of sixty-four frames from a 0:02:22 [H:MM:SS] video.\n\nFrame from 00:00:\nFrame from 00:03:\nFrame from 00:05:\nFrame from 00:07:\nFrame from 00:09:\nFrame from 00:12:\nFrame from 00:14:\nFrame from 00:16:\nFrame from 00:18:\nFrame from 00:21:\nFrame from 00:23:\nFrame from 00:25:\nFrame from 00:27:\nFrame from 00:29:\nFrame from 00:32:\nFrame from 00:34:\nFrame from 00:36:\nFrame from 00:38:\nFrame from 00:41:\nFrame from 00:43:\nFrame from 00:45:\nFrame from 00:47:\nFrame from 00:50:\nFrame from 00:52:\nFrame from 00:54:\nFrame from 00:56:\nFrame from 00:58:\nFrame from 01:01:\nFrame from 01:03:\nFrame from 01:05:\nFrame from 01:07:\nFrame from 01:10:\nFrame from 01:12:\nFrame from 01:14:\nFrame from 01:16:\nFrame from 01:18:\nFrame from 01:21:\nFrame from 01:23:\nFrame from 01:25:\nFrame from 01:27:\nFrame from 01:30:\nFrame from 01:32:\nFrame from 01:34:\nFrame from 01:36:\nFrame from 01:39:\nFrame from 01:41:\nFrame from 01:43:\nFrame

In [28]:
prompt="""
Extract the item names from this invoice. please do not include unit price and total amount. provide the name in list
"""


     

In [29]:
image = Image.open('invoice.png').convert('RGB')

In [30]:
messages= [{"role": "user","content": [{"type": "text", "text": prompt},{"type": "image" ,"path": image},]},]

In [31]:
inputs = processor.apply_chat_template( messages, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt", ).to(model.device, torch.bfloat16) 


In [32]:
generated_ids = model.generate(**inputs, do_sample=False, max_new_tokens=150) 

In [33]:
generated_texts = processor.batch_decode( generated_ids, skip_special_tokens=True, )

In [39]:
print(generated_texts[0])

User: 
Extract the item names from this invoice. please do not include unit price and total amount. provide the name in list






Assistant: The item names from the invoice are:
- Front and rear brake cables
- New set of pedal arms
- Labor 3 hours
